# Fantasy Premier League (FPL) Advisor Neural Network Model Builder

The purpose of this notebook is to train a neural network that can predict the expected points for specific player and fixture combination. Currently, the training data is based on [fpl-data](https://github.com/177arc/fpl-data) which contains a rolling window of fixtures from this and the past season.

# Installation
To get started, run the following command to install all required dependencies.

In [ ]:
#!pip install -q -r requirements.txt
#!pip install -q -r requirements_nn.txt

# Import requirements
Here we import all external and local modulues.

In [ ]:
import pandas as pd, os, sys
from fplpandas import FPLPandas
from datadict.jupyter import DataDict

import tensorflow as tf
from tensorflow import feature_column
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import matplotlib.pyplot as plt

# Load local modules
sys.path.append(os.getcwd())
from data import get_df
from nn import *

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 100)

# Set variables
This section sets all important global variables.

In [ ]:
data_url = 'https://s3.eu-west-2.amazonaws.com/fpl-test.177arc.net/v1/latest/'

# Load pre-processed data
This section loads data sets generated by the [fpl-data](https://github.com/177arc/fpl-data) lambda function and made available via the S3 bucket specified in the `data_url` variable.

In [ ]:
players_fixture_team_eps_ext = get_df(url=f'{data_url}players_fixture_team_eps_ext.csv', index=['Player Code', 'Season', 'Game Week'])
players_fixture_team_eps_ext.head(5)

## Create training and test datasets 

In [ ]:
train_df, test_df = (players_fixture_team_eps_ext
    .reset_index()
#    [['Fixture Total Points', 'Field Position', 'Total Points To Fixture', 'Is Home?', 'Fixtures Played To Fixture', 'Opp Team FDR']]
    [lambda df: df['Fixture Minutes Played'] > 0]    
    [['Fixture Total Points', 'Field Position', 'Total Points To Fixture', 'Fixtures Played To Fixture', 'Opp Team FDR']]
#    .assign(**{'Total Points': lambda df: df['Fixture Total Points']}
    .dropna(how='any', axis=0)
    .pipe(nn_split, frac=0.8))
train_ds = train_df.pipe(nn_prep_ds, 'Fixture Total Points')
test_ds = test_df.pipe(nn_prep_ds, 'Fixture Total Points')

## Create feature columns

In [ ]:
feature_columns_cats = {'field_position': ['GK', 'DEF', 'MID', 'FWD']}

feature_columns = []
for col, spec in train_ds.element_spec[0].items():
    if spec.dtype in [tf.bool, tf.float64]:
        feature_columns.append(feature_column.numeric_column(col))
        
    if col in feature_columns_cats.keys():
        field_pos = feature_column.categorical_column_with_vocabulary_list(col, feature_columns_cats[col])
        field_pos_one_hot = feature_column.indicator_column(field_pos)
        feature_columns.append(field_pos_one_hot)        

## Create model
Here we create a neural network with four layers. Although ultimately the mean 

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.DenseFeatures(feature_columns, dtype='float64'),
  layers.Dense(4, activation='relu', dtype='float64'),
  layers.Dense(3, activation='relu', dtype='float64'),
  layers.Dense(1, dtype='float64')
])

model.compile(loss='mse',
                optimizer=tf.keras.optimizers.RMSprop(0.001),
                metrics=['mse', 'mae'])

## Train model

In [ ]:
# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

train_history = model.fit(train_ds,
          validation_data=test_ds,
          epochs=60,
          callbacks=[early_stop])

## Evaluate model

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)
plotter.plot({'Early Stopping': train_history}, metric = 'mse')
plt.ylabel('MSE [Fixture]')

In [ ]:
test_predictions = model.predict(test_ds).flatten()
a = plt.axes(aspect='equal')
plt.scatter(test_df['Fixture Total Points'], test_predictions)
plt.xlabel('True Values [Fixture Total Points]')
plt.ylabel('Predictions [Fixture Total Points]')
lims = [-3, 20]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

In [ ]:
test_df['Predicted'] = test_predictions
test_df

## Save model

In [ ]:
model.save('models/expected_points')